# Исследование данных об инвестиции венчурных фондов в компании-стартапы

#### Исследование данных об инвестиции венчурных фондов в компании-стартапы

**Задача 1**  
Посчитайте, сколько компаний закрылось.

In [ ]:
SELECT COUNT(status)
FROM company
WHERE status LIKE 'closed';

**Задача 2**  

Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы `company`. Отсортируйте таблицу по убыванию значений в поле `funding_total`.

In [ ]:
SELECT 
       SUM(funding_total)
FROM company
WHERE country_code LIKE 'USA' AND category_code LIKE 'news'
GROUP BY name
ORDER BY SUM(funding_total) DESC;

**Задача 3**

Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

In [ ]:
SELECT SUM(price_amount)
FROM acquisition
WHERE term_code LIKE 'cash'
AND acquired_at BETWEEN '2011-01-01' AND '2013-12-31';

**Задача 4**  
Отобразите имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на `'Silver'`.

In [ ]:
SELECT first_name,
       last_name,
       twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%';

**Задача 5**  
Выведите на экран всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку `'money'`, а фамилия начинается на `'K'`.

In [ ]:
SELECT *
FROM people
WHERE twitter_username LIKE '%money%' AND last_name LIKE 'K%';

**Задача 6**  

Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

In [ ]:
SELECT country_code,
       SUM(funding_total)
FROM company
GROUP BY country_code
ORDER BY SUM(funding_total) DESC;

**Задача 7**  

Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.
Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

In [ ]:
SELECT CAST(funded_at AS date) ,
       MIN(raised_amount),
       MAX(raised_amount) 
FROM funding_round
GROUP BY CAST(funded_at AS date)
HAVING MIN(raised_amount) <> 0 AND MIN(raised_amount) <> MAX(raised_amount);

**Задача 8**  
Создайте поле с категориями:
Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию `high_activity`.
Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию `middle_activity`.
Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию `low_activity`.
Отобразите все поля таблицы `fund` и новое поле с категориями.

In [ ]:
SELECT *,
       CASE
           WHEN invested_companies >= 100 THEN 'high_activity'
           WHEN invested_companies >= 20 AND invested_companies < 100 THEN 'middle_activity'
           WHEN invested_companies < 20 THEN 'low_activity'
       END
FROM fund;

**Задача 9**  

Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

In [ ]:
SELECT CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity,
       ROUND(AVG(investment_rounds))
FROM fund
GROUP BY activity
ORDER BY ROUND(AVG(investment_rounds));

**Задача 10**  
Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы.  
Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. Выгрузите десять самых активных стран-инвесторов.  
Отсортируйте таблицу по среднему количеству компаний от большего к меньшему, а затем по коду страны в лексикографическом порядке.

In [ ]:
SELECT country_code,
      AVG(invested_companies),
      MAX(invested_companies),
      MIN(invested_companies)
FROM fund
WHERE EXTRACT (YEAR FROM CAST(founded_at AS DATE)) BETWEEN 2010 AND 2012
GROUP BY country_code
HAVING MIN(invested_companies) <> 0
ORDER BY AVG(invested_companies) DESC
LIMIT 10;

**Задача 11**  

Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

In [ ]:
SELECT p.first_name,
       p.last_name,
       e.instituition
FROM people as p
LEFT JOIN education AS e ON e.person_id = p.id

**Задача 12**  
Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

In [ ]:
SELECT c.name,
       COUNT(DISTINCT e.instituition)
FROM company AS c
Inner JOIN people as p ON p.company_id=c.id
Inner JOIN education as e ON p.id=e.person_id
GROUP BY c.name
ORDER BY COUNT(DISTINCT e.instituition) DESC
LIMIT 5;

**Задача 13**  
Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

In [ ]:
SELECT DISTINCT c.name
FROM COMPANY AS c
JOIN funding_round AS fr ON fr.company_id = c.id

WHERE c.status LIKE 'closed'
      AND fr.is_first_round = 1 and fr.is_last_round = 1;

**Задача 14**  
  Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

In [ ]:
SELECT DISTINCT p.id
FROM people as p
LEFT JOIN company AS c ON p.company_id = c.id
WHERE C.name IN (SELECT DISTINCT c.name
FROM COMPANY AS c
JOIN funding_round AS fr ON fr.company_id = c.id

WHERE c.status LIKE 'closed'
      AND fr.is_first_round = 1 and fr.is_last_round = 1);

**Задача 15**  

Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

In [ ]:
SELECT DISTINCT p.id,
       e.instituition
FROM people as p
LEFT JOIN company AS c ON p.company_id = c.id
JOIN education AS e on e.person_id=p.id
WHERE C.name IN (SELECT DISTINCT c.name
FROM COMPANY AS c
JOIN funding_round AS fr ON fr.company_id = c.id

WHERE c.status LIKE 'closed'
      AND fr.is_first_round = 1 and fr.is_last_round = 1);

**Задача 16**  
Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

In [ ]:
SELECT DISTINCT p.id,
       COUNT(e.instituition)
FROM people as p
LEFT JOIN company AS c ON p.company_id = c.id
JOIN education AS e on e.person_id=p.id
WHERE C.name IN (SELECT DISTINCT c.name
FROM COMPANY AS c
JOIN funding_round AS fr ON fr.company_id = c.id

WHERE c.status LIKE 'closed'
      AND fr.is_first_round = 1 and fr.is_last_round = 1)
GROUP BY p.id;

**Задача 17**  
Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

In [ ]:
SELECT AVG(t.count)
FROM(SELECT DISTINCT p.id,
       COUNT(e.instituition) as count
FROM people as p
LEFT JOIN company AS c ON p.company_id = c.id
JOIN education AS e on e.person_id=p.id
WHERE C.name IN (SELECT DISTINCT c.name
FROM COMPANY AS c
JOIN funding_round AS fr ON fr.company_id = c.id

WHERE c.status LIKE 'closed'
      AND fr.is_first_round = 1 and fr.is_last_round = 1)
GROUP BY p.id) AS t;

**Задача 18**  

Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Facebook*.
*(сервис, запрещённый на территории РФ)

In [ ]:
SELECT AVG(t.count)
FROM(SELECT DISTINCT p.id,
       COUNT(e.instituition) as count
FROM people as p
LEFT JOIN company AS c ON p.company_id = c.id
JOIN education AS e on e.person_id=p.id
WHERE C.name = 'Facebook'
GROUP BY p.id) AS t;

**Задача 19**  
Составьте таблицу из полей:
 - `name_of_fund` — название фонда;
 - `name_of_company` — название компании;
 - `amount` — сумма инвестиций, которую привлекла компания в раунде.
В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

In [ ]:
SELECT f.name AS name_of_fund,
       c.name AS name_of_company,
       fr.amount AS amount
FROM fund AS f
RIGHT JOIN investment AS i ON  f.id = i.fund_id
RIGHT JOIN company AS c ON i.company_id = c.id
iNNER JOIN (SELECT id,
         raised_amount AS amount
  FROM funding_round
  WHERE CAST(funded_at AS date) BETWEEN '2012-01-01' AND '2013-12-31') AS fr ON fr.id = i.funding_round_id
WHERE c.milestones > 6 ;

**Задача 20**  
Выгрузите таблицу, в которой будут такие поля:
- название компании-покупателя;
- сумма сделки;
- название компании, которую купили;
- сумма инвестиций, вложенных в купленную компанию;
- доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.  

Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы.
Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями. 

In [ ]:
SELECT c1.name AS acquiring_company,
       a.price_amount AS price_amount,
       c2.name AS acquired_company,
       c2.funding_total AS funding_total,
       ROUND(a.price_amount / c2.funding_total) AS PR
FROM acquisition AS a 
LEFT JOIN company AS c1 ON a.acquiring_company_id = c1.id
LEFT JOIN company AS c2 ON a.acquired_company_id = c2.id
WHERE  a.price_amount !=0 AND c2.funding_total !=0
ORDER BY price_amount DESC, acquired_company
LIMIT 10;

**Задача 21**  
Выгрузите таблицу, в которую войдут названия компаний из категории `social`, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования. 

In [ ]:
SELECT c.name AS name,
       EXTRACt(MONTH FROM CAST(fr.funded_at AS date)) AS month
FROM company AS c
LEFT JOIN funding_round AS fr ON fr.company_id=c.id
WHERE c.category_code LIKE 'social' AND fr.raised_amount <> 0
     AND EXTRACt(YEAR FROM CAST(fr.funded_at AS date)) BETWEEN 2010 AND 2013

**Задача 22**  
Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
- номер месяца, в котором проходили раунды;
- количество уникальных названий фондов из США, которые инвестировали в этом месяце;
- количество компаний, купленных за этот месяц;
- общая сумма сделок по покупкам в этом месяце.

In [ ]:
t1 AS (
SELECT EXTRACT(MONTH FROM CAST(fr.funded_at AS date)) AS month,
       COUNT(DISTINCT f.name) AS funds
FROM funding_round AS fr
JOIN investment AS i ON i.funding_round_id = fr.id
JOIN fund AS f ON f.id = i.fund_id   
WHERE CAST(funded_at AS date) BETWEEN '2010-01-01' AND '2013-12-31'
     AND f.country_code LIKE 'USA'
GROUP BY EXTRACT(MONTH FROM CAST(funded_at AS date))
),
t2 AS (
SELECT EXTRACT(MONTH FROM CAST(acquired_at AS date)) AS month,
       COUNT(acquired_company_id) AS acquired_company,
       SUM(price_amount) AS amount
FROM acquisition
WHERE CAST(acquired_at AS date) BETWEEN '2010-01-01' AND '2013-12-31'
GROUP BY EXTRACT(MONTH FROM CAST(acquired_at AS date))
)
SELECT t1.month,
       t1.funds,
       t2.acquired_company,
       t2.amount
FROM t1
JOIN t2 ON t1.month = t2.month;

**Задача 23**  
Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.


In [ ]:
WITH
y1 AS (
SELECT country_code,
      AVG(funding_total) AS year_2011
FROM company
WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2011
GROUP BY country_code
),
y2 AS (
SELECT country_code,
       AVG(funding_total) AS year_2012
FROM company
WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2012
GROUP BY country_code
),
y3 AS (
SELECT country_code,
       AVG(funding_total) AS year_2013
FROM company
WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2013
GROUP BY country_code
)
SELECT y1.country_code,
       y1.year_2011,
       y2.year_2012,
       y3.year_2013
FROM y1
JOIN y2 ON y1.country_code = y2.country_code
JOIN y3 ON y1.country_code = y3.country_code
ORDER BY y1.year_2011 DESC;